## <a id = '0'> Índice </a>

* [**Entorno**](#1)  
   * [Librerías](#1d1)  
   * [Funciones](#1d2)  
   * [Constantes](#1d3)

* [**Lectura de datos**](#2)


## <a id = '1'> Entorno </a>
[índice](#0)

### <a id = '1d1'> Librerías </a>

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import  LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, classification_report, make_scorer, confusion_matrix
from scipy.stats import ks_2samp

import matplotlib.pyplot as plt

import math
import itertools

import joblib
# from config import data_folder

In [2]:
os.chdir("../")

In [ ]:
MODEL_PATH = "output/models/V1/" 

### <a id = '1d2'> Funciones </a>

In [4]:
from src.utils import *

## <a id = '2'> Lectura de datos </a>
[índice](#0)

In [5]:
train_data = pd.read_csv("output/features/transform/train_features_pca.csv")
test_data = pd.read_csv("output/features/transform/test_features_pca.csv")

In [6]:
#Columnas que no vamos a usar en el modelado
skip_columns = ["patient", "label"]
le = LabelEncoder()

#Generamos las características y la variable objetivo 
X_train = train_data.drop(columns = skip_columns)
y_train = le.fit_transform(train_data["label"])

X_test = test_data.drop(columns = skip_columns)
y_test = le.fit_transform(test_data["label"])


In [170]:
train_data.shape

(1728, 287)

In [7]:
recall_macro = make_scorer(recall_score, average='macro') #Todas las clases tienen el mismo peso
f1_score_macro = make_scorer(f1_score, average='macro') #Todas las clases tienen el mismo peso

## Logistic Regression

In [91]:
# Estimador
lr = LogisticRegression(penalty='l2', 
                        C=1e5, 
                        solver='lbfgs', 
                        multi_class='multinomial',
                        random_state = 42)

# Parámetros
params_lr = {
    'penalty': ['l2'],
    'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
}
       
# Grid Search
grid_lr = GridSearchCV(estimator=lr,
                       param_grid=params_lr,
                       scoring=recall_macro,
                       cv=5,

                       verbose=1,
                       n_jobs=-1)

In [92]:
grid_lr.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Users/leongaray/Desktop/Estancia MCD/MCD_EKG/MCD_venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/leongaray/Desktop/Estancia MCD/MCD_EKG/MCD_venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/leongaray/Desktop/Estancia MCD/MCD_EKG/MCD_venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/leonga

GridSearchCV(cv=5,
             estimator=LogisticRegression(C=100000.0, multi_class='multinomial',
                                          random_state=42),
             n_jobs=-1,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10],
                         'penalty': ['l2']},
             scoring=make_scorer(recall_score, response_method='predict', average=macro),
             verbose=1)

In [93]:
#Pronóstico
y_pred_grid_lr = grid_lr.predict(X_test)
y_pred_prob_grid_lr = grid_lr.predict_proba(X_test)

In [161]:
grid_lr.best_params_

{'C': 0.001, 'penalty': 'l2'}

In [ ]:
print(params_to_markdown(grid_lr.best_params_))

In [162]:
grid_lr.best_score_

0.5440724405239241

In [163]:
grid_lr.best_estimator_

LogisticRegression(C=0.001, multi_class='multinomial', random_state=42)

In [138]:
joblib.dump(grid_lr, MODEL_PATH + "LR/logistic_regression_model.pkl")

LR_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_lr,
})

LR_probas = pd.DataFrame(y_pred_prob_grid_lr, columns=[f"proba_clase_{c}" for c in grid_lr.classes_])
LR_results = pd.concat([LR_preds, LR_probas], axis=1)
LR_results.to_csv(MODEL_PATH + "LR/predictions.csv", index=False)


In [94]:
table_metrics(y_test,y_pred_grid_lr, y_pred_prob_grid_lr)

,metric,value
0,accuracy,0.527083
1,precision_weighted,0.528874
2,recall_weighted,0.527083
3,f1_weighted,0.527638
4,roc_auc_ovr,0.790475
5,log_loss,1.068772
6,gini_normalized,0.580949
7,ks_test_clase_0,0.408333
8,ks_test_clase_1,0.525000
9,ks_test_clase_2,0.388889


In [103]:
confusion_matrix(y_test, y_pred_grid_lr)

array([[58, 20, 17, 25],
       [20, 68, 25,  7],
       [18, 17, 59, 26],
       [23,  6, 23, 68]])

In [95]:
print(classification_report(y_test, y_pred_grid_lr))

              precision    recall  f1-score   support

           0       0.49      0.48      0.49       120
           1       0.61      0.57      0.59       120
           2       0.48      0.49      0.48       120
           3       0.54      0.57      0.55       120

    accuracy                           0.53       480
   macro avg       0.53      0.53      0.53       480
weighted avg       0.53      0.53      0.53       480



In [154]:
genera_metricas_markdown(y_test,y_pred_grid_lr, y_pred_prob_grid_lr)

|    | metric             |    value |
|---:|:-------------------|---------:|
|  0 | accuracy           | 0.527083 |
|  1 | precision_weighted | 0.528874 |
|  2 | recall_weighted    | 0.527083 |
|  3 | f1_weighted        | 0.527638 |
|  4 | roc_auc_ovr        | 0.790475 |
|  5 | log_loss           | 1.06877  |
|  6 | gini_normalized    | 0.580949 |
|  7 | ks_test_clase_0    | 0.408333 |
|  8 | ks_test_clase_1    | 0.525    |
|  9 | ks_test_clase_2    | 0.388889 |
| 10 | ks_test_clase_3    | 0.494444 |


|              |   precision |   recall |   f1-score |    support |
|:-------------|------------:|---------:|-----------:|-----------:|
| 0            |    0.487395 | 0.483333 |   0.485356 | 120        |
| 1            |    0.612613 | 0.566667 |   0.588745 | 120        |
| 2            |    0.475806 | 0.491667 |   0.483607 | 120        |
| 3            |    0.539683 | 0.566667 |   0.552846 | 120        |
| accuracy     |    0.527083 | 0.527083 |   0.527083 |   0.527083 |
| macro avg    

In [157]:
cm_lr = confusion_matrix(y_test, y_pred_grid_lr)
df_cm_lr = pd.DataFrame(cm_lr,
                         index = [f"Real {label}" for label in grid_lr.classes_],
                         columns = [f"Pred {label}" for label in grid_lr.classes_])
print(df_cm_lr.to_markdown())


|        |   Pred 0 |   Pred 1 |   Pred 2 |   Pred 3 |
|:-------|---------:|---------:|---------:|---------:|
| Real 0 |       58 |       20 |       17 |       25 |
| Real 1 |       20 |       68 |       25 |        7 |
| Real 2 |       18 |       17 |       59 |       26 |
| Real 3 |       23 |        6 |       23 |       68 |


## Random Forest Classifier

In [98]:
#Estimador
rfc = RandomForestClassifier(random_state = 42, 
                             n_jobs = -1, 
                             bootstrap = True)

#Parámetros
params_rfc = {'n_estimators': [100, 350, 500],
             'max_features': ['log2', 'sqrt'],
             'max_depth': [5, 10, 20],
             'min_samples_split': [2, 10, 30],
             'min_samples_leaf': [2, 10, 30]}

#Grid Search
grid_rfc = GridSearchCV(estimator=rfc,
                       param_grid=params_rfc,
                       scoring=recall_macro,
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

In [99]:
grid_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 20],
                         'max_features': ['log2', 'sqrt'],
                         'min_samples_leaf': [2, 10, 30],
                         'min_samples_split': [2, 10, 30],
                         'n_estimators': [100, 350, 500]},
             scoring=make_scorer(recall_score, response_method='predict', average=macro),
             verbose=1)

In [164]:
grid_rfc.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 500}

In [ ]:
print(params_to_markdown(grid_rfc.best_params_))

In [165]:
grid_rfc.best_score_

0.5122026196204224

In [166]:
grid_rfc.best_estimator_

RandomForestClassifier(max_depth=10, min_samples_leaf=2, n_estimators=500,
                       n_jobs=-1, random_state=42)

In [110]:
#Pronóstico
y_pred_grid_rfc = grid_rfc.predict(X_test)
y_pred_prob_grid_rfc = grid_rfc.predict_proba(X_test)

In [139]:
joblib.dump(grid_rfc, MODEL_PATH + "RFC/random_forest_model.pkl")

RFC_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_rfc,
})

RFC_probas = pd.DataFrame(y_pred_prob_grid_rfc, columns=[f"proba_clase_{c}" for c in grid_rfc.classes_])
RFC_results = pd.concat([RFC_preds, RFC_probas], axis=1)
RFC_results.to_csv(MODEL_PATH + "RFC/predictions.csv", index=False)


In [120]:
table_metrics(y_test,y_pred_grid_rfc, y_pred_prob_grid_rfc)

,metric,value
0,accuracy,0.483333
1,precision_weighted,0.481121
2,recall_weighted,0.483333
3,f1_weighted,0.481030
4,roc_auc_ovr,0.751476
5,log_loss,1.241261
6,gini_normalized,0.502951
7,ks_test_clase_0,0.302778
8,ks_test_clase_1,0.522222
9,ks_test_clase_2,0.327778


In [159]:
confusion_matrix(y_test, y_pred_grid_rfc)

array([[48, 22, 20, 30],
       [23, 70, 21,  6],
       [25, 21, 45, 29],
       [27,  9, 15, 69]])

In [113]:
print(classification_report(y_test, y_pred_grid_rfc))

              precision    recall  f1-score   support

           0       0.39      0.40      0.40       120
           1       0.57      0.58      0.58       120
           2       0.45      0.38      0.41       120
           3       0.51      0.57      0.54       120

    accuracy                           0.48       480
   macro avg       0.48      0.48      0.48       480
weighted avg       0.48      0.48      0.48       480



In [153]:
genera_metricas_markdown(y_test,y_pred_grid_rfc, y_pred_prob_grid_rfc)

|    | metric             |    value |
|---:|:-------------------|---------:|
|  0 | accuracy           | 0.483333 |
|  1 | precision_weighted | 0.481121 |
|  2 | recall_weighted    | 0.483333 |
|  3 | f1_weighted        | 0.48103  |
|  4 | roc_auc_ovr        | 0.751476 |
|  5 | log_loss           | 1.24126  |
|  6 | gini_normalized    | 0.502951 |
|  7 | ks_test_clase_0    | 0.302778 |
|  8 | ks_test_clase_1    | 0.522222 |
|  9 | ks_test_clase_2    | 0.327778 |
| 10 | ks_test_clase_3    | 0.45     |


|              |   precision |   recall |   f1-score |    support |
|:-------------|------------:|---------:|-----------:|-----------:|
| 0            |    0.390244 | 0.4      |   0.395062 | 120        |
| 1            |    0.57377  | 0.583333 |   0.578512 | 120        |
| 2            |    0.445545 | 0.375    |   0.40724  | 120        |
| 3            |    0.514925 | 0.575    |   0.543307 | 120        |
| accuracy     |    0.483333 | 0.483333 |   0.483333 |   0.483333 |
| macro avg    

In [160]:
cm_rfc = confusion_matrix(y_test, y_pred_grid_rfc)
df_cm_rfc = pd.DataFrame(cm_rfc,
                         index = [f"Real {label}" for label in grid_rfc.classes_],
                         columns = [f"Pred {label}" for label in grid_rfc.classes_])
print(df_cm_rfc.to_markdown())


|        |   Pred 0 |   Pred 1 |   Pred 2 |   Pred 3 |
|:-------|---------:|---------:|---------:|---------:|
| Real 0 |       48 |       22 |       20 |       30 |
| Real 1 |       23 |       70 |       21 |        6 |
| Real 2 |       25 |       21 |       45 |       29 |
| Real 3 |       27 |        9 |       15 |       69 |


# Gradient Boosting Classifier

In [108]:
#Estimador
gb = GradientBoostingClassifier(learning_rate=0.05, 
                                subsample=0.5, 
                                max_depth=6, 
                                n_estimators=10,
                                 random_state=42,
                                )

#Parámetros
params_gb = {'n_estimators': [1,10,100], 
             'learning_rate' : [0.01,0.05,0.1],
             'subsample' : [0.1,0.5,1.0], 
             'max_depth': [5,10,20],
             'min_samples_split': [2, 10, 30],
             'min_samples_leaf': [2, 10, 30],
             'max_features': ['log2', 'sqrt']}

#Grid Search
grid_gb = GridSearchCV(estimator=gb,
                       param_grid=params_gb,
                       scoring=recall_macro,
                       cv = 5,
                       verbose=1,
                       n_jobs=-1)

In [109]:
#Entrenamiento
grid_gb.fit(X_train, y_train)

Fitting 5 folds for each of 1458 candidates, totalling 7290 fits


GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=0.05,
                                                  max_depth=6, n_estimators=10,
                                                  random_state=42,
                                                  subsample=0.5),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [5, 10, 20],
                         'max_features': ['log2', 'sqrt'],
                         'min_samples_leaf': [2, 10, 30],
                         'min_samples_split': [2, 10, 30],
                         'n_estimators': [1, 10, 100],
                         'subsample': [0.1, 0.5, 1.0]},
             scoring=make_scorer(recall_score, response_method='predict', average=macro),
             verbose=1)

In [167]:
grid_gb.best_params_

{'learning_rate': 0.01,
 'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 30,
 'min_samples_split': 2,
 'n_estimators': 100,
 'subsample': 1.0}

In [ ]:
print(params_to_markdown(grid_gb.best_params_))

In [168]:
grid_gb.best_score_

0.5046444800855386

In [169]:
grid_gb.best_estimator_

GradientBoostingClassifier(learning_rate=0.01, max_depth=20,
                           max_features='sqrt', min_samples_leaf=30,
                           random_state=42)

In [114]:
#Predicciones
y_pred_grid_gb = grid_gb.predict(X_test)
y_pred_prob_grid_gb = grid_gb.predict_proba(X_test)

In [140]:
joblib.dump(grid_rfc, MODEL_PATH + "GB/gradient_boosting_model.pkl")

GB_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_gb,
})

GB_probas = pd.DataFrame(y_pred_prob_grid_gb, columns=[f"proba_clase_{c}" for c in grid_gb.classes_])
GB_results = pd.concat([GB_preds, GB_probas], axis=1)
GB_results.to_csv(MODEL_PATH + "GB/predictions.csv", index=False)


In [ ]:
table_metrics(y_test,y_pred_grid_gb, y_pred_prob_grid_gb)

,metric,value
0,accuracy,0.500000
1,precision_weighted,0.501884
2,recall_weighted,0.500000
3,f1_weighted,0.500224
4,roc_auc_ovr,0.751817
5,log_loss,1.257561
6,gini_normalized,0.503634
7,ks_test_clase_0,0.350000
8,ks_test_clase_1,0.480556
9,ks_test_clase_2,0.344444


In [116]:
confusion_matrix(y_test, y_pred_grid_gb)

array([[57, 20, 20, 23],
       [26, 65, 24,  5],
       [21, 19, 50, 30],
       [28,  7, 17, 68]])

In [146]:
print(classification_report(y_test, y_pred_grid_gb))

              precision    recall  f1-score   support

           0       0.43      0.47      0.45       120
           1       0.59      0.54      0.56       120
           2       0.45      0.42      0.43       120
           3       0.54      0.57      0.55       120

    accuracy                           0.50       480
   macro avg       0.50      0.50      0.50       480
weighted avg       0.50      0.50      0.50       480



In [152]:
genera_metricas_markdown(y_test,y_pred_grid_gb, y_pred_prob_grid_gb)

|    | metric             |    value |
|---:|:-------------------|---------:|
|  0 | accuracy           | 0.5      |
|  1 | precision_weighted | 0.501884 |
|  2 | recall_weighted    | 0.5      |
|  3 | f1_weighted        | 0.500224 |
|  4 | roc_auc_ovr        | 0.751817 |
|  5 | log_loss           | 1.25756  |
|  6 | gini_normalized    | 0.503634 |
|  7 | ks_test_clase_0    | 0.35     |
|  8 | ks_test_clase_1    | 0.480556 |
|  9 | ks_test_clase_2    | 0.344444 |
| 10 | ks_test_clase_3    | 0.480556 |


|              |   precision |   recall |   f1-score |   support |
|:-------------|------------:|---------:|-----------:|----------:|
| 0            |    0.431818 | 0.475    |   0.452381 |     120   |
| 1            |    0.585586 | 0.541667 |   0.562771 |     120   |
| 2            |    0.45045  | 0.416667 |   0.4329   |     120   |
| 3            |    0.539683 | 0.566667 |   0.552846 |     120   |
| accuracy     |    0.5      | 0.5      |   0.5      |       0.5 |
| macro avg    |    0.

In [158]:
cm_gb = confusion_matrix(y_test, y_pred_grid_gb)
df_cm_gb = pd.DataFrame(cm_gb,
                         index = [f"Real {label}" for label in grid_gb.classes_],
                         columns = [f"Pred {label}" for label in grid_gb.classes_])
print(df_cm_gb.to_markdown())

|        |   Pred 0 |   Pred 1 |   Pred 2 |   Pred 3 |
|:-------|---------:|---------:|---------:|---------:|
| Real 0 |       57 |       20 |       20 |       23 |
| Real 1 |       26 |       65 |       24 |        5 |
| Real 2 |       21 |       19 |       50 |       30 |
| Real 3 |       28 |        7 |       17 |       68 |


## Naive Bayes

In [180]:
gnb = GaussianNB()

params_gnb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
}
#Grid Search
grid_gnb = GridSearchCV(estimator=gnb,
                       param_grid=params_gnb,
                       scoring=recall_macro,
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

In [181]:
#Entrenamiento
grid_gnb.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05,
                                           0.0001, 0.001]},
             scoring=make_scorer(recall_score, response_method='predict', average=macro),
             verbose=1)

In [182]:
grid_gnb.best_params_

{'var_smoothing': 1e-09}

In [ ]:
print(params_to_markdown(grid_gnb.best_params_))

In [183]:
grid_gnb.best_score_

0.36405372894947885

In [185]:
#Predicciones
y_pred_grid_gnb = grid_gnb.predict(X_test)
y_pred_prob_grid_gnb = grid_gnb.predict_proba(X_test)

In [186]:
joblib.dump(grid_gnb, MODEL_PATH + "GNB/Gaussian_Naive_Bayes_model.pkl")

GNB_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_gnb,
})

GNB_probas = pd.DataFrame(y_pred_prob_grid_gnb, columns=[f"proba_clase_{c}" for c in grid_gnb.classes_])
GNB_results = pd.concat([GNB_preds, GNB_probas], axis=1)
GNB_results.to_csv(MODEL_PATH + "GNB/predictions.csv", index=False)


In [187]:
table_metrics(y_test,y_pred_grid_gnb, y_pred_prob_grid_gnb)

,metric,value
0,accuracy,0.335417
1,precision_weighted,0.374984
2,recall_weighted,0.335417
3,f1_weighted,0.287205
4,roc_auc_ovr,0.616916
5,log_loss,6.404771
6,gini_normalized,0.233831
7,ks_test_clase_0,0.186111
8,ks_test_clase_1,0.266667
9,ks_test_clase_2,0.163889


In [188]:
confusion_matrix(y_test, y_pred_grid_gnb)

array([[47, 65,  5,  3],
       [25, 88,  6,  1],
       [32, 65, 12, 11],
       [35, 61, 10, 14]])

In [189]:
print(classification_report(y_test, y_pred_grid_gnb))

              precision    recall  f1-score   support

           0       0.34      0.39      0.36       120
           1       0.32      0.73      0.44       120
           2       0.36      0.10      0.16       120
           3       0.48      0.12      0.19       120

    accuracy                           0.34       480
   macro avg       0.37      0.34      0.29       480
weighted avg       0.37      0.34      0.29       480



In [190]:
genera_metricas_markdown(y_test,y_pred_grid_gnb, y_pred_prob_grid_gnb)

|    | metric             |    value |
|---:|:-------------------|---------:|
|  0 | accuracy           | 0.335417 |
|  1 | precision_weighted | 0.374984 |
|  2 | recall_weighted    | 0.335417 |
|  3 | f1_weighted        | 0.287205 |
|  4 | roc_auc_ovr        | 0.616916 |
|  5 | log_loss           | 6.40477  |
|  6 | gini_normalized    | 0.233831 |
|  7 | ks_test_clase_0    | 0.186111 |
|  8 | ks_test_clase_1    | 0.266667 |
|  9 | ks_test_clase_2    | 0.163889 |
| 10 | ks_test_clase_3    | 0.216667 |


|              |   precision |   recall |   f1-score |    support |
|:-------------|------------:|---------:|-----------:|-----------:|
| 0            |    0.338129 | 0.391667 |   0.362934 | 120        |
| 1            |    0.315412 | 0.733333 |   0.441103 | 120        |
| 2            |    0.363636 | 0.1      |   0.156863 | 120        |
| 3            |    0.482759 | 0.116667 |   0.187919 | 120        |
| accuracy     |    0.335417 | 0.335417 |   0.335417 |   0.335417 |
| macro avg    

In [191]:
cm_gnb = confusion_matrix(y_test, y_pred_grid_gnb)
df_cm_gnb = pd.DataFrame(cm_gnb,
                         index = [f"Real {label}" for label in grid_gnb.classes_],
                         columns = [f"Pred {label}" for label in grid_gnb.classes_])
print(df_cm_gnb.to_markdown())

|        |   Pred 0 |   Pred 1 |   Pred 2 |   Pred 3 |
|:-------|---------:|---------:|---------:|---------:|
| Real 0 |       47 |       65 |        5 |        3 |
| Real 1 |       25 |       88 |        6 |        1 |
| Real 2 |       32 |       65 |       12 |       11 |
| Real 3 |       35 |       61 |       10 |       14 |


## XGB Classifier

In [8]:
xgb_base = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=5000,
    eval_metric="merror",
    objective="multi:softprob",
    # early_stopping_rounds=50,
    
)

xgb_base.fit(X_train, y_train,
             eval_set=[(X_test, y_test)],             
             verbose = 1)

# xgb_n_estimator = xgb_base.best_iteration

[0]	validation_0-merror:0.56250
[1]	validation_0-merror:0.55625
[2]	validation_0-merror:0.54792
[3]	validation_0-merror:0.56667
[4]	validation_0-merror:0.56875
[5]	validation_0-merror:0.55833
[6]	validation_0-merror:0.55833
[7]	validation_0-merror:0.54167
[8]	validation_0-merror:0.55208
[9]	validation_0-merror:0.53750
[10]	validation_0-merror:0.54167
[11]	validation_0-merror:0.54167
[12]	validation_0-merror:0.54167
[13]	validation_0-merror:0.53750
[14]	validation_0-merror:0.54375
[15]	validation_0-merror:0.53958
[16]	validation_0-merror:0.53542
[17]	validation_0-merror:0.52917
[18]	validation_0-merror:0.53333
[19]	validation_0-merror:0.52917
[20]	validation_0-merror:0.52917
[21]	validation_0-merror:0.53333
[22]	validation_0-merror:0.53125
[23]	validation_0-merror:0.52708
[24]	validation_0-merror:0.52708
[25]	validation_0-merror:0.52500
[26]	validation_0-merror:0.51458
[27]	validation_0-merror:0.51667
[28]	validation_0-merror:0.51875
[29]	validation_0-merror:0.51458
[30]	validation_0-me

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='merror',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5000, n_jobs=None,
              num_parallel_tree=None, ...)

In [9]:
#Predicciones
y_pred_grid_xgb_b = xgb_base.predict(X_test)
y_pred_prob_grid_xgb_b = xgb_base.predict_proba(X_test)

In [10]:
table_metrics(y_test,y_pred_grid_xgb_b, y_pred_prob_grid_xgb_b)

,metric,value
0,accuracy,0.514583
1,precision_weighted,0.516245
2,recall_weighted,0.514583
3,f1_weighted,0.515084
4,roc_auc_ovr,0.762303
5,log_loss,1.477503
6,gini_normalized,0.524606
7,ks_test_clase_0,0.333333
8,ks_test_clase_1,0.494444
9,ks_test_clase_2,0.380556


In [13]:
joblib.dump(xgb_base, MODEL_PATH + "XGB/XGB_base_model.pkl")

XGB_base_preds = pd.DataFrame({
    "patient" : test_data["patient"],
    "label" : test_data["label"],
    "y_true" : y_test,
    "pred" : y_pred_grid_xgb_b,
})

XGB_base_probas = pd.DataFrame(y_pred_prob_grid_xgb_b, columns=[f"proba_clase_{c}" for c in xgb_base.classes_])
XGB_base_results = pd.concat([XGB_base_preds, XGB_base_probas], axis=1)
XGB_base_results.to_csv(MODEL_PATH + "XGB/predictions_base.csv", index=False)


In [15]:
genera_metricas_markdown(y_test,y_pred_grid_xgb_b, y_pred_prob_grid_xgb_b)

|    | metric             |    value |
|---:|:-------------------|---------:|
|  0 | accuracy           | 0.514583 |
|  1 | precision_weighted | 0.516245 |
|  2 | recall_weighted    | 0.514583 |
|  3 | f1_weighted        | 0.515084 |
|  4 | roc_auc_ovr        | 0.762303 |
|  5 | log_loss           | 1.4775   |
|  6 | gini_normalized    | 0.524606 |
|  7 | ks_test_clase_0    | 0.333333 |
|  8 | ks_test_clase_1    | 0.494444 |
|  9 | ks_test_clase_2    | 0.380556 |
| 10 | ks_test_clase_3    | 0.505556 |


|              |   precision |   recall |   f1-score |    support |
|:-------------|------------:|---------:|-----------:|-----------:|
| 0            |    0.445378 | 0.441667 |   0.443515 | 120        |
| 1            |    0.603604 | 0.558333 |   0.580087 | 120        |
| 2            |    0.48     | 0.5      |   0.489796 | 120        |
| 3            |    0.536    | 0.558333 |   0.546939 | 120        |
| accuracy     |    0.514583 | 0.514583 |   0.514583 |   0.514583 |
| macro avg    

In [16]:
cm_xgb_base = confusion_matrix(y_test, y_pred_grid_xgb_b)
df_cm_xgb_base = pd.DataFrame(cm_xgb_base,
                         index = [f"Real {label}" for label in xgb_base.classes_],
                         columns = [f"Pred {label}" for label in xgb_base.classes_])
print(df_cm_xgb_base.to_markdown())

|        |   Pred 0 |   Pred 1 |   Pred 2 |   Pred 3 |
|:-------|---------:|---------:|---------:|---------:|
| Real 0 |       53 |       22 |       14 |       31 |
| Real 1 |       22 |       67 |       26 |        5 |
| Real 2 |       22 |       16 |       60 |       22 |
| Real 3 |       22 |        6 |       25 |       67 |


In [34]:
print(params_to_markdown(xgb_base.get_params()))

| parametro          | valor          |
|:-------------------|:---------------|
| objective          | multi:softprob |
| enable_categorical | False          |
| eval_metric        | merror         |
| learning_rate      | 0.1            |
| missing            | nan            |
| n_estimators       | 5000           |


In [ ]:
#Transformación de datos para entrenar modelo
xgb_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
xgb_dmatrix_test = xgb.DMatrix(data=X_test, label=y_test)

#Parámetros
params_xgb = {'learning_rate': [0.01, 0.1, 0.5],
              'n_estimators' : [100, 350, 500],
              'subsample' : [0.3, 0.5, 0.9 ],
              'max_depth': [5, 10, 20],
              'min_child_weight': [1, 5, 10],
              'colsample_bytree': [0.3, 0.5, 0.9],
}

#Estimador
xgb = xgb.XGBClassifier()

#Grid Search
grid_xgb = GridSearchCV(estimator=xgb,
                       param_grid=params_xgb,
                       scoring=recall_macro,
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

In [ ]:
grid_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


/Users/leongaray/Desktop/Estancia MCD/MCD_EKG/MCD_venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 